#**Tutorial #2: Exploring Pedestrian Count Data: Temporal and Spatial Insights**
This tutorial uses the City of Sydney’s multi-year pedestrian counts to explore how walking varies across time and place. You’ll combine site metadata with survey records to build tidy datasets, map the network of count locations, and visualise trends by year, season and day type. The focus is on interpreting patterns, not just plotting them, so that results can inform street design, active transport planning, and urban re-activation.

The tutorial complements our paper, **"Association between land use features and changes in walking patterns from pre-pandemic to post-pandemic: A case study of City of Sydney (2013–2023)"** published in the Journal of Transport and Land Use (2024). Read it for the modeling approach (time-series clustering, regression) and policy implications that go beyond this exploratory notebook: https://doi.org/10.5198/jtlu.2024.2511

**Who is this for?** Students, analysts and practitioners in transport/urban planning who want a hands-on, reproducible workflow in Python/Colab for turning raw counts into decision-ready insights.

**Learning outcomes:**
*   Understand how to structure count data (one value per site–year–season/period).
*   Compare sites (spatial variation) and years (temporal change) without double-counting seasons.
*   Use maps and time-series to frame planning questions (e.g., CBD vs neighbourhood centres, weekday vs weekend, recovery since COVID).

**Data:**
*   [Walking_count_sites.csv](https://drive.google.com/uc?id=1PbiAg6_FKTq01sYKuVVZmgBbXcBFPT-e) — location and attributes of each counting site.
*   [Walking_count_surveys.csv](https://drive.google.com/uc?id=1EONKhXMFxGY3Cs_y7YI-11fAQOG0Zw9p) — survey records by site, year, season, and day type.

Both files are provided with this tutorial via Google Drive links. Alternatively, they can be downloaded from the [City of Sydney Open Data Hub](https://www.cityofsydney.nsw.gov.au/public-health-safety-programs/walking-counts).

# **Setup & load data**

In this section, we fetch the two CSVs, load them into DataFrames and do a quick sanity check on shapes and columns. This ensures you know what units you’re working in (survey-day totals vs hourly) before any analysis.

In [1]:
# Cell 1 — Setup & load data from Google Drive

# 1) Imports and small utilities
import pandas as pd
import re

# 2) (One-time) install + download from public Google Drive
#    If you've already uploaded the CSVs to Colab, you can skip this block.
!pip -q install gdown
import gdown

# Enable ipywidgets in Colab
!pip -q install ipywidgets==8.1.1
from google.colab import output
output.enable_custom_widget_manager()

# Google Drive file IDs (from the share links)
SURVEYS_ID = "1PbiAg6_FKTq01sYKuVVZmgBbXcBFPT-e"
SITES_ID   = "1EONKhXMFxGY3Cs_y7YI-11fAQOG0Zw9p"

surveys_path = "Walking_count_surveys.csv"
sites_path   = "Walking_count_sites.csv"

# Download (won't re-download if files already exist with same name)
gdown.download(id=SURVEYS_ID, output=surveys_path, quiet=False)
gdown.download(id=SITES_ID,   output=sites_path,   quiet=False)

# 3) Load CSVs
surveys = pd.read_csv(surveys_path)
sites   = pd.read_csv(sites_path)

# 4) Quick structural summary
print("Surveys shape:", surveys.shape)
print("Sites shape  :", sites.shape)

print("\nSurveys columns:\n", surveys.columns.tolist())
print("\nSites columns:\n",   sites.columns.tolist())

# 5) Peek at the first few rows
display(surveys.head(5))
display(sites.head(5))

# 6) Light sanity checks (handy for students)
#    - detect hourly columns (Time_0600 ... Time_2300)
hourly_cols = [c for c in surveys.columns if re.match(r"^Time_\d{4}$", c)]
print(f"\nDetected {len(hourly_cols)} hourly columns (e.g., first five):", hourly_cols[:5])


Downloading...
From: https://drive.google.com/uc?id=1PbiAg6_FKTq01sYKuVVZmgBbXcBFPT-e
To: /content/Walking_count_surveys.csv
100%|██████████| 917k/917k [00:00<00:00, 112MB/s]
Downloading...
From: https://drive.google.com/uc?id=1EONKhXMFxGY3Cs_y7YI-11fAQOG0Zw9p
To: /content/Walking_count_sites.csv
100%|██████████| 10.4k/10.4k [00:00<00:00, 18.3MB/s]

Surveys shape: (4933, 27)
Sites shape  : (120, 6)

Surveys columns:
 ['OBJECTID', 'SiteID', 'Location', 'Description', 'Period', 'Year', 'Month', 'TotalCount', 'DateFilter', 'Time_0600', 'Time_0700', 'Time_0800', 'Time_0900', 'Time_1000', 'Time_1100', 'Time_1200', 'Time_1300', 'Time_1400', 'Time_1500', 'Time_1600', 'Time_1700', 'Time_1800', 'Time_1900', 'Time_2000', 'Time_2100', 'Time_2200', 'Time_2300']

Sites columns:
 ['X', 'Y', 'OBJECTID', 'Site_ID', 'Location', 'SiteDescription']


,OBJECTID,SiteID,Location,Description,Period,Year,Month,TotalCount,DateFilter,Time_0600,...,Time_1400,Time_1500,Time_1600,Time_1700,Time_1800,Time_1900,Time_2000,Time_2100,Time_2200,Time_2300
0,6760,2,Botany Road,Between Bourke Street and Hansard Street,weekday,2013,Spring,3504,2013 - Weekday - October,186,...,78,96,258,462,372,294,234,174,36,24
1,6761,2,Botany Road,Between Bourke Street and Hansard Street,weekend,2013,Spring,834,2013 - Weekend - October,24,...,30,24,54,90,60,18,42,24,30,6
2,6762,2,Botany Road,Between Bourke Street and Hansard Street,weekday,2014,Autumn,3300,2014 - Weekday - May,168,...,150,156,306,522,198,138,132,84,24,24
3,6763,2,Botany Road,Between Bourke Street and Hansard Street,weekday,2014,Spring,2844,2014 - Weekday - October,78,...,126,264,282,540,162,162,54,54,42,18
4,6764,2,Botany Road,Between Bourke Street and Hansard Street,weekend,2014,Autumn,1134,2014 - Weekend - May,54,...,114,126,78,24,48,30,12,12,18,12


,X,Y,OBJECTID,Site_ID,Location,SiteDescription
0,333848.1805,6.246666e+06,1,2,Botany Road,Between Bourke Street and Hansard Street
1,331528.3390,6.247511e+06,2,3,King Street,Between Whitehorse Street and Newman Street
2,335060.3495,6.250342e+06,3,4,William Street,Between Crown Street and Palmer Street
3,332445.5838,6.248375e+06,4,5,City Road,Between Carillon Avenue and Forbes Street
4,333326.7975,6.249206e+06,5,6,Broadway,Between Buckland Street and Abercrombie Street



Detected 18 hourly columns (e.g., first five): ['Time_0600', 'Time_0700', 'Time_0800', 'Time_0900', 'Time_1000']


# **Clean and join datasets**

Here, we standardise key fields (SiteID, Year) and merge surveys with site metadata. A light validation step flags unmatched site IDs (data quality), years covered and period labels (season, weekday/weekend), missing coordinates (limits for mapping). This gives you a single “analysis-ready” table with one row per observation and the attributes needed for grouping.

In [2]:
# Cell 2 — Clean, harmonise, and join datasets

import pandas as pd
import re

# Copy to avoid mutating the originals
sites_df   = sites.copy()
surveys_df = surveys.copy()

# 1) Tidy column names (strip spaces)
sites_df.columns   = [c.strip() for c in sites_df.columns]
surveys_df.columns = [c.strip() for c in surveys_df.columns]

# 2) Ensure key types
if "Site_ID" in sites_df.columns:
    sites_df["Site_ID"] = pd.to_numeric(sites_df["Site_ID"], errors="coerce")

if "SiteID" in surveys_df.columns:
    surveys_df["SiteID"] = pd.to_numeric(surveys_df["SiteID"], errors="coerce")

if "Year" in surveys_df.columns:
    surveys_df["Year"] = pd.to_numeric(surveys_df["Year"], errors="coerce").astype("Int64")

# 3) Detect hourly columns (Time_HHMM)
hourly_cols = [c for c in surveys_df.columns if re.match(r"^Time_\d{4}$", c)]
hourly_cols = sorted(hourly_cols)

print(f"Detected {len(hourly_cols)} hourly columns. Example: {hourly_cols[:5]}")

# 4) Merge surveys with site metadata
#    (rename Site_ID -> SiteID in the sites table for a clean join key)
sites_renamed = sites_df.rename(columns={"Site_ID": "SiteID"})
try:
    merged = surveys_df.merge(
        sites_renamed,
        on="SiteID",
        how="left",
        validate="m:1"   # each survey row maps to at most one site row
    )
except Exception as e:
    print("Merge validation warning:", e)
    merged = surveys_df.merge(sites_renamed, on="SiteID", how="left")

# 5) Quick sanity checks
n_sites_ref      = sites_df["Site_ID"].nunique() if "Site_ID" in sites_df.columns else None
n_sites_observed = surveys_df["SiteID"].nunique()
year_min = int(merged["Year"].dropna().min()) if "Year" in merged.columns and merged["Year"].notna().any() else None
year_max = int(merged["Year"].dropna().max()) if "Year" in merged.columns and merged["Year"].notna().any() else None
periods  = sorted(merged["Period"].dropna().unique().tolist()) if "Period" in merged.columns else []

print("\n=== Sanity checks ===")
print("Unique sites in reference table :", n_sites_ref)
print("Unique sites appearing in surveys:", n_sites_observed)
print("Year range in surveys            :", (year_min, year_max))
print("Periods present                  :", periods)

# Unmatched SiteIDs (surveys that didn't find a site row)
ref_siteids     = set(sites_df["Site_ID"].unique()) if "Site_ID" in sites_df.columns else set()
survey_siteids  = set(surveys_df["SiteID"].dropna().unique())
unmatched_ids   = sorted(list(survey_siteids - ref_siteids))
print(f"\nUnmatched SiteIDs (in surveys but not in sites): {len(unmatched_ids)}")
print(unmatched_ids[:20])

# How many merged rows are missing coordinates?
missing_coords_mask = merged[["X","Y"]].isna().any(axis=1) if {"X","Y"}.issubset(merged.columns) else pd.Series(False, index=merged.index)
share_missing_coords = missing_coords_mask.mean()
print(f"Rows missing coordinates after merge: {share_missing_coords:.2%}")

# Peek at results
print("\nMerged preview:")
display(merged.head(8))


Detected 18 hourly columns. Example: ['Time_0600', 'Time_0700', 'Time_0800', 'Time_0900', 'Time_1000']

=== Sanity checks ===
Unique sites in reference table : 120
Unique sites appearing in surveys: 130
Year range in surveys            : (2013, 2025)
Periods present                  : ['weekday', 'weekend']

Unmatched SiteIDs (in surveys but not in sites): 10
[np.int64(123), np.int64(124), np.int64(132), np.int64(155), np.int64(215), np.int64(219), np.int64(222), np.int64(232), np.int64(235), np.int64(239)]
Rows missing coordinates after merge: 2.23%

Merged preview:


,OBJECTID_x,SiteID,Location_x,Description,Period,Year,Month,TotalCount,DateFilter,Time_0600,...,Time_1900,Time_2000,Time_2100,Time_2200,Time_2300,X,Y,OBJECTID_y,Location_y,SiteDescription
0,6760,2,Botany Road,Between Bourke Street and Hansard Street,weekday,2013,Spring,3504,2013 - Weekday - October,186,...,294,234,174,36,24,333848.1805,6.246666e+06,1.0,Botany Road,Between Bourke Street and Hansard Street
1,6761,2,Botany Road,Between Bourke Street and Hansard Street,weekend,2013,Spring,834,2013 - Weekend - October,24,...,18,42,24,30,6,333848.1805,6.246666e+06,1.0,Botany Road,Between Bourke Street and Hansard Street
2,6762,2,Botany Road,Between Bourke Street and Hansard Street,weekday,2014,Autumn,3300,2014 - Weekday - May,168,...,138,132,84,24,24,333848.1805,6.246666e+06,1.0,Botany Road,Between Bourke Street and Hansard Street
3,6763,2,Botany Road,Between Bourke Street and Hansard Street,weekday,2014,Spring,2844,2014 - Weekday - October,78,...,162,54,54,42,18,333848.1805,6.246666e+06,1.0,Botany Road,Between Bourke Street and Hansard Street
4,6764,2,Botany Road,Between Bourke Street and Hansard Street,weekend,2014,Autumn,1134,2014 - Weekend - May,54,...,30,12,12,18,12,333848.1805,6.246666e+06,1.0,Botany Road,Between Bourke Street and Hansard Street
5,6765,2,Botany Road,Between Bourke Street and Hansard Street,weekend,2014,Spring,780,2014 - Weekend - October,24,...,54,36,0,12,24,333848.1805,6.246666e+06,1.0,Botany Road,Between Bourke Street and Hansard Street
6,6766,2,Botany Road,Between Bourke Street and Hansard Street,weekday,2015,Autumn,5802,2015 - Weekday - March,402,...,246,84,96,12,24,333848.1805,6.246666e+06,1.0,Botany Road,Between Bourke Street and Hansard Street
7,6767,2,Botany Road,Between Bourke Street and Hansard Street,weekday,2015,Spring,1236,2015 - Weekday - October,42,...,24,36,42,144,96,333848.1805,6.246666e+06,1.0,Botany Road,Between Bourke Street and Hansard Street


# **Visualise the location of pedestrian count sites on a map**
Here, we map all sites and size/colour them by the latest available year. This is a quick supply-side picture, where we measure pedestrian activity, and how volumes are distributed in 2025.

**Interpretation tips:**
*   Large circles in clusters suggest high activity corridors or destinations
*   Overlaps/voids may reveal gaps in the monitoring network, not just low demand.
*   Use popups to relate volumes to street function (transit spine vs local main street).

In [3]:
# Cell — Folium map with circle size & colour based on latest counts (no clustering)

# Requirements (run once per runtime)
!pip -q install folium pyproj branca

import numpy as np
import pandas as pd
import folium
from pyproj import Transformer
import branca.colormap as bcm

# --- Inputs expected from earlier cells ---
# - sites_df: reference sites table with columns ["Site_ID","X","Y","Location","SiteDescription", ...]
# - merged:    surveys joined to sites, with at least ["SiteID","Year","TotalCount"] present

# 1) Latest-year totals per site
if {"SiteID", "Year", "TotalCount"}.issubset(merged.columns):
    latest_year = int(merged["Year"].dropna().max())
    latest = (
        merged.loc[merged["Year"] == latest_year]
              .groupby("SiteID", as_index=False)["TotalCount"].sum()
              .rename(columns={"TotalCount": "LatestYearTotal"})
    )
else:
    raise ValueError("Merged dataframe must include columns: SiteID, Year, TotalCount")

# 2) Convert projected X/Y → WGS84 (lon/lat)
#    Sydney common CRSs:
#      EPSG:7856 (GDA2020 / MGA Zone 56)  ← default
#      EPSG:28356 (GDA94  / MGA Zone 56)
EPSG_SITES = 7856  # change to 28356 if points look offset

sites_xy = sites_df.dropna(subset=["X", "Y"]).copy()
transformer = Transformer.from_crs(f"EPSG:{EPSG_SITES}", "EPSG:4326", always_xy=True)
lon, lat = transformer.transform(sites_xy["X"].values, sites_xy["Y"].values)
sites_xy["lon"] = lon
sites_xy["lat"] = lat

# 3) Join latest totals to sites
mdf = sites_xy.rename(columns={"Site_ID": "SiteID"}).merge(latest, on="SiteID", how="left")

# Handle missing counts
mdf["LatestYearTotal"] = mdf["LatestYearTotal"].fillna(0)

# 4) Visual encodings: radius & colour from counts
vmax = float(mdf["LatestYearTotal"].max()) if len(mdf) else 1.0
vmin = float(mdf["LatestYearTotal"].min()) if len(mdf) else 0.0
if vmax <= 0:
    vmax = 1.0

# radius: ensure visibility even for small counts
def scale_radius(v, vmin=vmin, vmax=vmax, min_px=4.0, max_px=22.0):
    # sqrt scaling for perceived area
    return float(min_px + (np.sqrt(v / vmax) * (max_px - min_px))) if vmax > 0 else float(min_px)

mdf["radius"] = mdf["LatestYearTotal"].apply(scale_radius)

# Colour map (Light blue to Dark purple) for better contrast on bright basemap
cmap = bcm.LinearColormap(
    colors=["#deebf7", "#9ecae1", "#3182bd", "#08519c", "#4a1486"],
    vmin=vmin, vmax=vmax
)


# 5) Build Folium map (no clustering)
center_lat = float(mdf["lat"].mean())
center_lon = float(mdf["lon"].mean())
m = folium.Map(location=[center_lat, center_lon], zoom_start=13, tiles="CartoDB positron")

def make_popup(row):
    sid = int(row["SiteID"])
    loc = row.get("Location", "")
    desc = row.get("SiteDescription", "")
    tot = int(row.get("LatestYearTotal", 0))
    html = f"""
    <div style='font-size:13px;'>
      <b>Site {sid}</b><br/>
      <b>Location:</b> {loc}<br/>
      <b>Description:</b> {desc}<br/>
      <b>{latest_year} total:</b> {tot:,}
    </div>
    """
    return folium.Popup(html, max_width=320)


# Dark background basemap
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=13,
    tiles="CartoDB dark_matter"   # dark basemap
)

# Add circles directly
for _, r in mdf.iterrows():
    value = float(r["LatestYearTotal"])
    color = cmap(value)
    folium.CircleMarker(
        location=[r["lat"], r["lon"]],
        radius=float(r["radius"]),
        color=color,            # stroke colour
        opacity=0.5,            # 50% transparent stroke
        fill=True,
        fill_color=color,       # fill colour mapped to counts
        fill_opacity=0.5,       # 50% transparent fill
        weight=1.0,
        popup=make_popup(r),
        tooltip=f"Site {int(r['SiteID'])} • {latest_year}: {int(value):,}"
    ).add_to(m)

# Add legend (colour scale)
cmap.caption = f"{latest_year} total pedestrians"
cmap.add_to(m)

m  # Display directly in Colab


# **Interactive time-series by site (Plotly)**

Pick a site and plot separate lines by season and day type. Each point is the average per seasonal survey day (if multiple days were observed).

**What this tells you:**
*   Stability vs volatility at a site across years,
*   Seasonal asymmetry (e.g., Autumn > Spring),
*   Day-type differences (weekday commute vs weekend leisure).


In [28]:
# Cell — Single-site time series (fast, no normalisation; seasons kept separate)

import pandas as pd
import plotly.express as px
import numpy as np

# ---- Pick your site here ----
SITE_ID = 43  # ← change this to the SiteID you want

# 0) Quick guard: make sure SITE_ID exists in merged
if "SiteID" not in merged.columns:
    raise ValueError("merged does not have a 'SiteID' column. Make sure you've run the merge cell.")
if SITE_ID not in set(merged["SiteID"].dropna().astype(int).tolist()):
    avail = sorted(set(merged["SiteID"].dropna().astype(int).tolist()))
    raise ValueError(f"SITE_ID {SITE_ID} not found. Try one of: {avail[:25]} ...")

# 1) Filter data for the chosen site
df = merged.loc[merged["SiteID"] == SITE_ID].dropna(subset=["Year", "TotalCount"]).copy()
df["Year"] = df["Year"].astype(int)

# 2) Average if multiple survey days exist within the same Year/Season/Period
group_keys = ["Year"]
has_month  = "Month"  in df.columns and df["Month"].notna().any()     # Season/month info
has_period = "Period" in df.columns and df["Period"].notna().any()    # Weekday/Weekend

if has_month:
    group_keys.append("Month")
if has_period:
    group_keys.append("Period")

site_ts = (
    df.groupby(group_keys, as_index=False)["TotalCount"]
      .mean()
      .sort_values(group_keys)
)

if site_ts.empty:
    raise ValueError(f"No time-series rows for SITE_ID {SITE_ID} after filtering.")

# 3) Nice title from site metadata (robust to missing values)
meta = sites_df.rename(columns={"Site_ID": "SiteID"})
meta_row = meta.loc[meta["SiteID"] == SITE_ID].head(1)

def get_cell(row, col):
    if len(row) == 0 or col not in row.columns:
        return ""
    val = row[col].iloc[0]
    return "" if (val is None or (isinstance(val, float) and np.isnan(val))) else str(val)

loc  = get_cell(meta_row, "Location").strip()
desc = get_cell(meta_row, "SiteDescription").strip()
title_suffix = " — ".join([t for t in [loc, desc] if t])

# 4) Optional ordering for Month/Season if present
if has_month:
    # Try to order by seasons first; otherwise keep the dataset's order
    season_order = [m for m in ["Autumn", "Spring", "Winter", "Summer"] if m in site_ts["Month"].unique()]
    if season_order:
        site_ts["Month"] = pd.Categorical(site_ts["Month"], categories=season_order, ordered=True)

# 5) Plot
fig = px.line(
    site_ts,
    x="Year",
    y="TotalCount",
    color=("Month" if has_month else None),
    facet_col=("Period" if has_period else None),
    markers=True,
    title=f"Site {SITE_ID}" + (f" — {title_suffix}" if title_suffix else ""),
    labels={"TotalCount": "Pedestrians (per seasonal survey day)"}
)

fig.update_yaxes(tickformat=",")
fig.update_layout(height=420, legend_title_text=("Season" if has_month else None))
fig.show()


# **Overall yearly mean & median across all sites and seasons**

Here, we compute yearly mean and median across all site–season–period observations (no seasonal summing). The mean reflects the network with weight from high-volume sites (CBD, stations). The median reflects the typical site.
A widening mean–median gap signals polarisation, strong growth at a few anchors while neighbourhood sites lag.

In [11]:
# Cell — Overall yearly mean & median across all sites and seasons (no seasonal summing, Plotly)
# Also shows a table of results.

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# --- 1) Build per-site/year/season/period values (NO seasonal summing) ---
df = merged.dropna(subset=["Year", "TotalCount"]).copy()
df["Year"] = df["Year"].astype(int)

group_keys = ["SiteID", "Year"]
if "Month" in df.columns and df["Month"].notna().any():
    group_keys.append("Month")    # keep Autumn/Spring separate
if "Period" in df.columns and df["Period"].notna().any():
    group_keys.append("Period")   # keep weekday/weekend separate

# Average if multiple survey days exist within the same site/year/season/period
seasonal = (
    df.groupby(group_keys, as_index=False)["TotalCount"]
      .mean()
)

# --- 2) Overall yearly summary (mean & median across all sites & seasons) ---
yearly_summary = (
    seasonal.groupby("Year", as_index=False)["TotalCount"]
            .agg(mean="mean", median="median", n_obs="count")
            .sort_values("Year")
)

# --- 3) Plot (Plotly) ---
long = yearly_summary.melt(
    id_vars=["Year", "n_obs"],
    value_vars=["mean", "median"],
    var_name="Metric",
    value_name="Value"
)
long["Metric"] = long["Metric"].map({"mean": "Mean", "median": "Median"})

fig = px.line(
    long,
    x="Year",
    y="Value",
    color="Metric",
    markers=True,
    color_discrete_map={"Mean": "#1f77b4", "Median": "#ff7f0e"},
    title="All Sites & Seasons — Yearly Mean and Median (per seasonal survey day)",
    labels={"Value": "Pedestrians", "Year": "Year", "Metric": ""}
)

# y-axis formatting & hover shows number of observations that year
fig.update_yaxes(tickformat=",")
fig.update_traces(
    hovertemplate=(
        "Year: %{x}<br>"
        "%{fullData.name}: %{y:,.0f}<br>"
        "Observations that year: %{customdata:,}"
        "<extra></extra>"
    ),
    customdata=long["n_obs"]
)
fig.update_layout(height=420)
fig.show()

# --- 4) Table (Plotly Table) ---
table_fig = go.Figure(
    data=[
        go.Table(
            header=dict(
                values=["Year", "Mean", "Median", "Observations"],
                fill_color="#f2f2f2",
                align="left"
            ),
            cells=dict(
                values=[
                    yearly_summary["Year"],
                    yearly_summary["mean"].map(lambda x: f"{x:,.0f}"),
                    yearly_summary["median"].map(lambda x: f"{x:,.0f}"),
                    yearly_summary["n_obs"].map(lambda x: f"{x:,}")
                ],
                align="left"
            )
        )
    ]
)
table_fig.update_layout(title="Yearly Mean & Median (table)", height=400)
table_fig.show()


# **Understanding the post-pandemic pedestrian activity recovery rate**

This section estimates how pedestrian activity has recovered each year relative to 2019. It:

*   Averages any multiple survey days within the same site–year–season–period (so each point is “per seasonal survey day”).
*   Computes the overall mean and median across all sites/seasons for each year.
*   Sets 2019 as the baseline (100%). For each year ≥2020 it calculates recovery % = value / 2019 × 100, plus year-over-year (YoY) changes in percentage points.
*   Outputs a line chart (mean & median recovery) and a table with the underlying numbers.

**Interpretation of the results**

Pedestrian activity fell to about half of 2019 levels in 2020–21, then climbed steadily through 2022–23. By 2024 the mean reached roughly three-quarters of 2019, and in 2025 the mean moved closer to ninety percent, while the median has hovered around two-thirds since 2023. This gap between mean and median indicates that a handful of high-volume locations have driven much of the rebound, whereas the typical site remains well below pre-COVID levels. (Note: 2025 has fewer observations, so interpret that uptick with some caution.)

**Takeaways points**

*   Recovery has been gradual, toward ~85–90% (mean) by 2025.
*   Uneven rebound: mean outpaces median, implying strong gains at a subset of busy sites while many others lag.
*   The typical site (median) remains around ~65% of 2019, suggesting the network hasn’t fully returned to pre-COVID activity.


In [13]:
# Cell — COVID recovery by year using 2019 as the ONLY baseline (Plotly + table)
# This computes recovery since 2020 relative to 2019:
# 1) Build per-site/year/season(/period) averages (NO seasonal summing).
# 2) For each year, compute overall MEAN and MEDIAN across all sites & seasons.
# 3) Set baseline = 2019 only (error if 2019 not present).
# 4) For each year >= 2020, recovery % = (metric / baseline_2019) * 100.
# 5) Compute YoY change in recovery (percentage points).
# 6) Plot Mean & Median recovery and show a summary table.

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# --- 1) Per-site/year/season(/period) values (NO seasonal summing) ---
df = merged.dropna(subset=["Year", "TotalCount"]).copy()
df["Year"] = df["Year"].astype(int)

group_keys = ["SiteID", "Year"]
if "Month" in df.columns and df["Month"].notna().any():
    group_keys.append("Month")    # keep Autumn/Spring separate
if "Period" in df.columns and df["Period"].notna().any():
    group_keys.append("Period")   # keep weekday/weekend separate

# Average if multiple survey days exist for the same site/year/season/period
seasonal = (
    df.groupby(group_keys, as_index=False)["TotalCount"]
      .mean()
)

# --- 2) Yearly overall MEAN & MEDIAN across all sites & seasons ---
yearly = (
    seasonal.groupby("Year", as_index=False)["TotalCount"]
            .agg(mean="mean", median="median", n_obs="count")
            .sort_values("Year")
)

if yearly.empty:
    raise ValueError("No yearly data found after grouping. Check 'merged' content.")

# --- 3) Baseline = 2019 ONLY ---
row_2019 = yearly[yearly["Year"] == 2019]
if row_2019.empty:
    available_years = ", ".join(map(str, yearly["Year"].tolist()))
    raise ValueError(f"2019 baseline not found in data. Available years: {available_years}")
baseline_mean   = float(row_2019["mean"].iloc[0])
baseline_median = float(row_2019["median"].iloc[0])

# guard against divide-by-zero
if baseline_mean == 0:   baseline_mean = np.nan
if baseline_median == 0: baseline_median = np.nan

# --- 4) Recovery % since 2020 (vs 2019) ---
yearly["recovery_mean_pct"]   = (yearly["mean"]   / baseline_mean)   * 100.0
yearly["recovery_median_pct"] = (yearly["median"] / baseline_median) * 100.0

rec = yearly[yearly["Year"] >= 2020].copy()
if rec.empty:
    raise ValueError("No years >= 2020 present to compute recovery.")

# --- 5) YoY change in recovery (percentage points) ---
rec = rec.sort_values("Year")
rec["yoy_change_mean_pp"]   = rec["recovery_mean_pct"].diff()
rec["yoy_change_median_pp"] = rec["recovery_median_pct"].diff()

# --- 6) Plot (Plotly) ---
long = rec.melt(
    id_vars=["Year", "n_obs"],
    value_vars=["recovery_mean_pct", "recovery_median_pct"],
    var_name="Metric",
    value_name="RecoveryPct"
)
long["Metric"] = long["Metric"].map({
    "recovery_mean_pct": "Mean recovery (%)",
    "recovery_median_pct": "Median recovery (%)"
})

fig = px.line(
    long,
    x="Year",
    y="RecoveryPct",
    color="Metric",
    markers=True,
    color_discrete_map={"Mean recovery (%)": "#1f77b4", "Median recovery (%)": "#ff7f0e"},
    title="Recovery since COVID (vs 2019 baseline) — All sites & seasons",
    labels={"RecoveryPct": "Recovery (% of 2019)", "Year": "Year", "Metric": ""}
)

# Add 100% baseline reference (2019)
fig.add_hline(y=100, line_dash="dash", line_color="#888",
              annotation_text="2019 baseline (100%)", annotation_position="top left")

# Hover shows n_obs
fig.update_traces(
    hovertemplate=(
        "Year: %{x}<br>"
        "%{fullData.name}: %{y:.1f}%<br>"
        "Observations that year: %{customdata:,}"
        "<extra></extra>"
    ),
    customdata=long["n_obs"]
)
fig.update_yaxes(ticksuffix="%", rangemode="tozero")
fig.update_layout(height=440, legend_title_text="")
fig.show()

# --- 7) Table (Plotly Table) ---
table_df = rec[["Year", "mean", "median", "recovery_mean_pct", "recovery_median_pct",
                "yoy_change_mean_pp", "yoy_change_median_pp", "n_obs"]].copy()

table_fig = go.Figure(
    data=[
        go.Table(
            header=dict(
                values=["Year", "Mean", "Median", "Mean recovery %", "Median recovery %",
                        "YoY change (mean, pp)", "YoY change (median, pp)", "Observations"],
                fill_color="#f2f2f2",
                align="left"
            ),
            cells=dict(
                values=[
                    table_df["Year"],
                    table_df["mean"].map(lambda x: f"{x:,.0f}"),
                    table_df["median"].map(lambda x: f"{x:,.0f}"),
                    table_df["recovery_mean_pct"].map(lambda x: f"{x:.1f}%"),
                    table_df["recovery_median_pct"].map(lambda x: f"{x:.1f}%"),
                    table_df["yoy_change_mean_pp"].map(lambda x: "" if pd.isna(x) else f"{x:+.1f}"),
                    table_df["yoy_change_median_pp"].map(lambda x: "" if pd.isna(x) else f"{x:+.1f}"),
                    table_df["n_obs"].map(lambda x: f"{x:,}")
                ],
                align="left"
            )
        )
    ]
)
table_fig.update_layout(title="Recovery summary table — Baseline = 2019 only", height=420)
table_fig.show()


# **2025 leaders & laggards (busiest sites and recovery vs 2019)**

This section ranks locations in four ways to give a clear end-state view of 2025 and how it compares with 2019. For fairness, it first computes a per-site, per-year value as the mean pedestrians per seasonal survey day: if a site has multiple survey days within the same year–season–period (e.g., Autumn/weekday), those are averaged, and then any seasons/periods observed that year are averaged again to one number per site–year. Using these site–year values it:

*   Lists the Top 10 busiest locations in 2025 (highest mean per survey day).
*   Lists the Lowest 10 busiest locations in 2025 (lowest mean per survey day).
*   Computes each site’s recovery in 2025 vs 2019 as 2025 / 2019 × 100%, then lists the Top 10 largest recovery and the Lowest 10 (slowest) recovery.
(Sites without a 2019 value are excluded from the recovery rankings.)

Each table shows the site ID and location, the 2025 mean, the 2019 mean (for context), and the recovery % where applicable. Use these lists to spot stand-out performers, laggards, and places where activity has bounced back the most—or least—since 2019.

**Interpretation of results**

In 2025 the busiest locations cluster along George Street (perhaps not so surprisingly), which dominates the top-10 list and also features prominently among the biggest recovery gains—suggesting strong CBD spine effects and event/retail pull. The largest recovery list also includes corridors like Botany Road and Young Street, indicating pockets outside George Street that have surged past their 2019 levels, most possibly due to new residential developments popin up in those areas. At the other end, the least busy and slowest recovery sites are mainly secondary or neighbourhood streets (e.g., South Dowling, Craigend, Arundel, St Johns, Elizabeth, Bayswater, Darlinghurst Road, Sydney Park), consistent with a slower rebound in local, non-core corridors.

In [14]:
# Cell — 2025 leaders & laggards: busiest sites and recovery vs 2019
# What this does (fast and fair):
# - Build per-site/year values as the MEAN "per seasonal survey day" (no seasonal summing).
#   If a site has multiple survey days within the same year/season/period, those are averaged first,
#   then averaged across seasons/periods to one value per site-year.
# - For YEAR_TARGET=2025, list:
#     1) Top 10 busiest sites (highest 2025 mean per seasonal survey day)
#     2) Bottom 10 busiest sites
# - Using BASELINE_YEAR=2019, compute site-level recovery in 2025:
#       Recovery% = 100 * (site_2025 / site_2019)
#   Then list:
#     3) Top 10 largest recovery in 2025
#     4) Bottom 10 (slowest) recovery in 2025
#
# Outputs four nicely formatted tables.

import pandas as pd
import numpy as np
import plotly.graph_objects as go

YEAR_TARGET   = 2025
BASELINE_YEAR = 2019

# ---------- 1) Build per-site/year "per seasonal survey day" values ----------
df = merged.dropna(subset=["SiteID","Year","TotalCount"]).copy()
df["Year"] = df["Year"].astype(int)

# Average duplicate survey days for the same site/year/season/period
keys = ["SiteID","Year"]
if "Month" in df.columns and df["Month"].notna().any():   keys.append("Month")
if "Period" in df.columns and df["Period"].notna().any(): keys.append("Period")

by_obs = (
    df.groupby(keys, as_index=False)["TotalCount"]
      .mean()
)

# Then collapse to one value per site-year by averaging across observed seasons/periods
site_year = (
    by_obs.groupby(["SiteID","Year"], as_index=False)["TotalCount"]
          .mean()
          .rename(columns={"TotalCount":"MeanPerSurveyDay"})
)

# Metadata for readability
meta = (
    sites_df.rename(columns={"Site_ID":"SiteID"})
            [["SiteID","Location","SiteDescription"]]
)

# ---------- 2) Extract target year and baseline (2019) ----------
target = site_year[site_year["Year"] == YEAR_TARGET][["SiteID","MeanPerSurveyDay"]] \
         .rename(columns={"MeanPerSurveyDay": f"Value_{YEAR_TARGET}"})
baseline = site_year[site_year["Year"] == BASELINE_YEAR][["SiteID","MeanPerSurveyDay"]] \
           .rename(columns={"MeanPerSurveyDay": f"Value_{BASELINE_YEAR}"})

stats = target.merge(baseline, on="SiteID", how="left")
stats = stats.merge(meta, on="SiteID", how="left")

# Recovery% vs 2019 (protect against divide-by-zero/missing baseline)
den = stats[f"Value_{BASELINE_YEAR}"].replace({0: np.nan})
stats["RecoveryPct_2025_vs_2019"] = (stats[f"Value_{YEAR_TARGET}"] / den) * 100.0

# ---------- 3) Build the four rankings ----------
def fmt_num(x):
    return "" if pd.isna(x) else f"{x:,.0f}"

def fmt_pct(x):
    return "" if pd.isna(x) else f"{x:.1f}%"

# a) Top 10 busiest in YEAR_TARGET
top_busiest = (stats.dropna(subset=[f"Value_{YEAR_TARGET}"])
                    .sort_values(f"Value_{YEAR_TARGET}", ascending=False)
                    .head(10)
                    .copy())

# b) Bottom 10 busiest in YEAR_TARGET
bottom_busiest = (stats.dropna(subset=[f"Value_{YEAR_TARGET}"])
                       .sort_values(f"Value_{YEAR_TARGET}", ascending=True)
                       .head(10)
                       .copy())

# c) Top 10 largest recovery in YEAR_TARGET vs 2019 (need baseline present)
top_recovery = (stats.dropna(subset=["RecoveryPct_2025_vs_2019"])
                     .sort_values("RecoveryPct_2025_vs_2019", ascending=False)
                     .head(10)
                     .copy())

# d) Bottom 10 (slowest) recovery in YEAR_TARGET vs 2019
bottom_recovery = (stats.dropna(subset=["RecoveryPct_2025_vs_2019"])
                        .sort_values("RecoveryPct_2025_vs_2019", ascending=True)
                        .head(10)
                        .copy())

# Helper to render a Plotly table
def show_table(df, title, show_baseline=False, show_recovery=False):
    cols = ["SiteID","Location"]
    values = [df["SiteID"],
              df["Location"].fillna("")]
    headers = ["Site ID","Location"]

    cols_val = [f"Value_{YEAR_TARGET}"]
    headers_val = [f"{YEAR_TARGET} mean per survey day"]
    values += [df[f"Value_{YEAR_TARGET}"].map(fmt_num)]
    headers += headers_val

    if show_baseline:
        values += [df.get(f"Value_{BASELINE_YEAR}", pd.Series([])).map(fmt_num)]
        headers += [f"{BASELINE_YEAR} mean per survey day"]
    if show_recovery:
        values += [df.get("RecoveryPct_2025_vs_2019", pd.Series([])).map(fmt_pct)]
        headers += [f"Recovery vs {BASELINE_YEAR}"]

    fig = go.Figure(data=[go.Table(
        header=dict(values=headers, fill_color="#f2f2f2", align="left"),
        cells=dict(values=values, align="left")
    )])
    fig.update_layout(title=title, height=420)
    fig.show()

# ---------- 4) Display ----------
show_table(top_busiest,   title=f"Top 10 busiest locations in {YEAR_TARGET}", show_baseline=True, show_recovery=True)
show_table(top_recovery,  title=f"Top 10 largest recovery in {YEAR_TARGET} vs {BASELINE_YEAR}", show_baseline=True, show_recovery=True)
show_table(bottom_busiest, title=f"Lowest 10 busiest locations in {YEAR_TARGET}", show_baseline=True, show_recovery=True)
show_table(bottom_recovery, title=f"Lowest 10 (slowest) recovery in {YEAR_TARGET} vs {BASELINE_YEAR}", show_baseline=True, show_recovery=True)


# **Side-by-side maps of 2025 “busiest” vs “recovery”**

This sections creates two interactive side-by-side maps to compare where pedestrian activity is highest now and where it has bounced back the most since 2019.

**Left map (current activity):** shows the Top 10 busiest locations in red and the Bottom 10 least busy in blue for 2025. Circle size reflects the 2025 pedestrian count (per survey day).

**Right map (recovery):** shows the Top 10 fastest-recovery locations in orange and the Bottom 10 slowest-recovery in purple, where circle size reflects % recovery vs 2019.

Use the layer toggles to show/hide the “top” and “bottom” groups on each map, switch between light/dark base maps, and hover/click a circle to see the site ID, name, 2025 value, 2019 value, and recovery %. Note that circle sizes are comparable within each map but not across the two maps (one scales by counts, the other by recovery %).

**Takeaway points:**
*   The busiest 2025 sites cluster tightly along the George Street/CBD spine, while the least-busy locations are more dispersed across neighbourhood streets and park edges, a very centralized pattern of activity.
*   The fastest recovery also concentrates in and around the CBD and a few inner-corridor sites (e.g., parts of Botany/Young/George), whereas slowest recovery dots peripheral or secondary streets like Darlinghurst Rd / South Dowling / Arundel / Sydney Park—places that remain well below 2019.
*   “Busiest” ≠ “most recovered.” Some high-volume CBD sites are star performers on both maps, but many lagging sites remain quiet and far from 2019 levels—pointing to uneven, corridor-specific rebound dynamics.

In [21]:
# Cell — Dual maps (no legends): left sized by 2025 count, right sized by % recovery

!pip -q install folium pyproj branca

import numpy as np
import pandas as pd
import folium
from folium.plugins import DualMap
from pyproj import Transformer

YEAR_TARGET   = 2025
BASELINE_YEAR = 2019
EPSG_SITES    = 7856  # switch to 28356 if markers look offset

# --- Expect merged & sites_df from earlier cells ---
if "merged" not in globals() or "sites_df" not in globals():
    raise ValueError("Please run the earlier cells that create `merged` and `sites_df`.")

# 1) Per site-year value = mean pedestrians per seasonal survey day
df = merged.dropna(subset=["SiteID","Year","TotalCount"]).copy()
df["Year"] = df["Year"].astype(int)

keys = ["SiteID","Year"]
if "Month" in df.columns and df["Month"].notna().any():   keys.append("Month")
if "Period" in df.columns and df["Period"].notna().any(): keys.append("Period")

by_obs = df.groupby(keys, as_index=False)["TotalCount"].mean()
site_year = (by_obs.groupby(["SiteID","Year"], as_index=False)["TotalCount"]
                  .mean()
                  .rename(columns={"TotalCount":"MeanPerSurveyDay"}))

# 2) 2025 value, 2019 baseline, recovery %
target   = site_year[site_year["Year"] == YEAR_TARGET][["SiteID","MeanPerSurveyDay"]] \
            .rename(columns={"MeanPerSurveyDay": f"Value_{YEAR_TARGET}"})
baseline = site_year[site_year["Year"] == BASELINE_YEAR][["SiteID","MeanPerSurveyDay"]] \
            .rename(columns={"MeanPerSurveyDay": f"Value_{BASELINE_YEAR}"})

stats = target.merge(baseline, on="SiteID", how="left")
stats["RecoveryPct_2025_vs_2019"] = (
    stats[f"Value_{YEAR_TARGET}"] / stats[f"Value_{BASELINE_YEAR}"].replace({0: np.nan})
) * 100.0

# 3) Join coords + a site name (fallbacks)
sites_xy = sites_df.rename(columns={"Site_ID":"SiteID"}).dropna(subset=["X","Y"]).copy()
name_candidates = [c for c in ["SiteName","Site_Name","Name","Label","StreetName","Location","SiteDescription"] if c in sites_xy.columns]
site_name_col = name_candidates[0] if name_candidates else None
sites_xy["SiteNameResolved"] = sites_xy[site_name_col] if site_name_col else ""

transformer = Transformer.from_crs(f"EPSG:{EPSG_SITES}", "EPSG:4326", always_xy=True)
lon, lat = transformer.transform(sites_xy["X"].values, sites_xy["Y"].values)
sites_xy["lon"] = lon
sites_xy["lat"] = lat

stats = stats.merge(sites_xy[["SiteID","lon","lat","SiteNameResolved"]], on="SiteID", how="left").dropna(subset=["lon","lat"])

# 4) Build lists
def top_n(df, col, n=10, asc=False):
    return df.dropna(subset=[col]).sort_values(col, ascending=asc).head(n).copy()

top_busiest     = top_n(stats, f"Value_{YEAR_TARGET}", n=10, asc=False)
bottom_busiest  = top_n(stats, f"Value_{YEAR_TARGET}", n=10, asc=True)
top_recovery    = top_n(stats, "RecoveryPct_2025_vs_2019", n=10, asc=False)
bottom_recovery = top_n(stats, "RecoveryPct_2025_vs_2019", n=10, asc=True)

# Shared size scales (left = 2025 counts, right = % recovery)
busy_vals = pd.concat([top_busiest[f"Value_{YEAR_TARGET}"], bottom_busiest[f"Value_{YEAR_TARGET}"]], ignore_index=True)
busy_vmin, busy_vmax = float(busy_vals.min()), float(busy_vals.max())

rec_vals = pd.concat([top_recovery["RecoveryPct_2025_vs_2019"], bottom_recovery["RecoveryPct_2025_vs_2019"]], ignore_index=True).dropna()
rec_vmin, rec_vmax = float(rec_vals.min()), float(rec_vals.max())

def radius_scale(v, vmin, vmax):
    if not np.isfinite(v) or not np.isfinite(vmin) or not np.isfinite(vmax) or vmax <= vmin:
        return 8.0
    # sqrt scaling for area perception
    return 6.0 + 10.0 * np.sqrt((v - vmin) / (vmax - vmin))

def popup_html(r):
    v2025 = r.get(f"Value_{YEAR_TARGET}", np.nan)
    v2019 = r.get(f"Value_{BASELINE_YEAR}", np.nan)
    rec   = r.get("RecoveryPct_2025_vs_2019", np.nan)
    return folium.Popup(
        f"<div style='font-size:13px'>"
        f"<b>Site {int(r['SiteID'])}</b><br/>"
        f"<b>Site name:</b> {r.get('SiteNameResolved','') or ''}<br/>"
        f"<b>{YEAR_TARGET} mean per survey day:</b> {'' if pd.isna(v2025) else f'{v2025:,.0f}'}<br/>"
        f"<b>{BASELINE_YEAR} mean per survey day:</b> {'' if pd.isna(v2019) else f'{v2019:,.0f}'}<br/>"
        f"<b>Recovery vs {BASELINE_YEAR}:</b> {'' if pd.isna(rec) else f'{rec:.1f}%'}"
        f"</div>", max_width=320
    )

def add_points(m, df, name, color, val_col, vmin, vmax):
    fg = folium.FeatureGroup(name=name, show=True)
    for _, r in df.iterrows():
        val = float(r[val_col]) if pd.notna(r[val_col]) else np.nan
        rad = radius_scale(val, vmin, vmax)
        folium.CircleMarker(
            location=[r["lat"], r["lon"]],
            radius=rad,
            color=color, opacity=0.6,
            fill=True, fill_color=color, fill_opacity=0.5,
            weight=1.0,
            tooltip=f"Site {int(r['SiteID'])} — {r.get('SiteNameResolved','') or ''}",
            popup=popup_html(r)
        ).add_to(fg)
    fg.add_to(m)

# 5) Dual maps + tiny gap between panes (no legends)
center_lat = float(stats["lat"].mean())
center_lon = float(stats["lon"].mean())
dual = DualMap(location=[center_lat, center_lon], tiles=None, zoom_start=13)

for mp in (dual.m1, dual.m2):
    folium.TileLayer("CartoDB positron", name="Light").add_to(mp)
    folium.TileLayer("CartoDB dark_matter", name="Dark").add_to(mp)

# LEFT: size ∝ 2025 count
COLOR_TOP_BUSY = "#e31a1c"  # red
COLOR_LOW_BUSY = "#1f78b4"  # blue
add_points(dual.m1, top_busiest,    "Top 10 busiest (2025)",     COLOR_TOP_BUSY, f"Value_{YEAR_TARGET}", busy_vmin, busy_vmax)
add_points(dual.m1, bottom_busiest, "Bottom 10 busiest (2025)",  COLOR_LOW_BUSY, f"Value_{YEAR_TARGET}", busy_vmin, busy_vmax)

# RIGHT: size ∝ % recovery
COLOR_TOP_REC = "#ff7f00"   # orange
COLOR_LOW_REC = "#6a3d9a"   # purple
add_points(dual.m2, top_recovery,    "Top 10 recovery (vs 2019)",   COLOR_TOP_REC, "RecoveryPct_2025_vs_2019", rec_vmin, rec_vmax)
add_points(dual.m2, bottom_recovery, "Bottom 10 recovery (vs 2019)",COLOR_LOW_REC, "RecoveryPct_2025_vs_2019", rec_vmin, rec_vmax)

# Layer controls (use to toggle Top/Bottom layers per pane)
folium.LayerControl(collapsed=False).add_to(dual.m1)
folium.LayerControl(collapsed=False).add_to(dual.m2)

# Small visual gap between the panes
left_id  = dual.m1.get_name()
right_id = dual.m2.get_name()
gap_css = f"""
<style>
  #{left_id}  {{ width: 49.6% !important; float:left !important; margin-right:0.8%; }}
  #{right_id} {{ width: 49.6% !important; float:left !important; }}
</style>
"""
dual.m1.get_root().html.add_child(folium.Element(gap_css))

dual  # display


# **Conclusion: what the data says about walking in the City of Sydney**

Across a decade of counts, the picture is consistent. Walking is strongly concentrated on the CBD spine, especially George Street, and post-COVID recovery is uneven. By 2025 the network mean is back to roughly 85–90% of 2019, while the median sits nearer ~65%, pointing to a corridor-led rebound driven by a small set of high-performing streets. A few non-core corridors (e.g., Botany and Young) now exceed 2019 levels, but many neighbourhood and park-edge streets remain subdued.

For planning, this suggests a two-track agenda. In the core, manage success—prioritise pedestrian capacity and safety (signal timing, crossing width, queue space, shade, event operations). In lagging areas, focus on reactivation and access—continuous footpath links, safer crossings to transit and schools, and place-management that rebuilds everyday walking demand.

**Some notes on methods and limitations to keep in mind:**
*   Unit of analysis: We work with per seasonal survey day values; do not sum seasons together.
*   Coverage effects. Some years have fewer sites or missing seasons/periods. Prefer relative metrics (e.g., % change) and always report the number of observations/sites (n).
*   Context change. Infrastructure works (e.g., light rail), construction, land-use churn, and events can shift counts independent of underlying demand.
*   Weather & timing. Counts are weather-sensitive; treat sharp year-to-year swings around outlier survey days with caution.
*   Equity lens. If the mean rises while the median stays flat, benefits are uneven. Consider targeting under-recovered neighbourhood centres.